## Milestone 6 - Querying - Most Popular Categories ##

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import when, col, count, row_number,expr

# Load cleaned data
df_pin = spark.read.format("delta").table("57e94de2a910_pin_cleaned")
df_geo = spark.read.format("delta").table("57e94de2a910_geo_cleaned")
df_user = spark.read.format("delta").table("57e94de2a910_user_cleaned")

# Link pin and geo data
pin_geo_link = df_pin.join(df_geo, df_pin.ind == df_geo.ind)

# Window function to partition by country
win = Window.partitionBy("country").orderBy(col("category_count").desc())

# Most popular category by country
pin_geo_link.groupBy("country", "category") \
.agg(count("category") \
.alias("category_count")) \
.withColumn("rank", row_number().over(win)) \
.filter(col("rank") == 1) \
.drop("rank") \
.show()


+--------------------+--------------+--------------+
|             country|      category|category_count|
+--------------------+--------------+--------------+
|         Afghanistan|     education|             6|
|             Albania|           art|             4|
|             Algeria|        quotes|            11|
|      American Samoa|       tattoos|             5|
|             Andorra|       tattoos|             5|
|              Angola|diy-and-crafts|             3|
|            Anguilla|diy-and-crafts|             2|
|Antarctica (the t...|       tattoos|             2|
| Antigua and Barbuda|     christmas|             5|
|           Argentina|       tattoos|             5|
|             Armenia|        beauty|             3|
|               Aruba|  mens-fashion|             5|
|           Australia|  mens-fashion|             2|
|             Austria|        travel|             2|
|          Azerbaijan|event-planning|             2|
|             Bahamas|        travel|         

## Milestone 6 - Querying - Most Popular Category Each Year ##

In [0]:
from pyspark.sql.functions import year, col, count

# Window function to partition by year
win =  Window.partitionBy("post_year").orderBy(col("category_count").desc())

pin_geo_link.withColumn("post_year", year("timestamp")) \
.filter(col("post_year") >= 2018) \
.filter(col("post_year") <= 2022) \
.groupBy("post_year", "category") \
.agg(count("category").alias("category_count")) \
.withColumn("rank", row_number().over(win)) \
.filter(col("rank") == 1) \
.drop("rank") \
.show()


+---------+--------------+--------------+
|post_year|      category|category_count|
+---------+--------------+--------------+
|     2018|     education|            12|
|     2019|diy-and-crafts|            12|
|     2020|       finance|            12|
|     2021|       tattoos|            11|
|     2022|     christmas|            14|
+---------+--------------+--------------+



## Milestone 6 - Querying - Users with most followers by country ##

In [0]:
# For each country find the user with the most followers

# Drop one of the duplicate 'ind' columns in the initial dataframe
pin_geo_link = pin_geo_link.drop(df_geo["ind"])

# Setup link to user dataframe and pin_geo_link dataframe
pin_geo_user = pin_geo_link.join(df_user, pin_geo_link["ind"] == df_user["ind"], "inner")

# Window function to partition by country
win = Window.partitionBy("country").orderBy(col("follower_count").desc())

# Perform the query
result_df = pin_geo_user \
.select("country", "poster_name", "follower_count") \
.withColumn("rank", row_number().over(win)) \
.filter(col("rank") == 1) \
.drop("rank") \

# Show new dataframe
result_df.show()

# Most followed user
win_global = Window.orderBy(col("follower_count").desc())

most_followed = result_df \
.withColumn("global_rank", row_number().over(win_global)) \
.filter(col("global_rank") == 1) \
.select("country", "follower_count") \

# Show new dataframe
most_followed.show()

+--------------------+--------------------+--------------+
|             country|         poster_name|follower_count|
+--------------------+--------------------+--------------+
|         Afghanistan|                9GAG|       3000000|
|             Albania|   The Minds Journal|       5000000|
|             Algeria|           YourTango|        942000|
|      American Samoa|         Mamas Uncut|       8000000|
|             Andorra|Teachers Pay Teac...|       1000000|
|              Angola|           Tastemade|       8000000|
|            Anguilla|Kristen | Lifesty...|         92000|
|Antarctica (the t...|          Refinery29|       1000000|
| Antigua and Barbuda|Country Living Ma...|       1000000|
|           Argentina|         Next Luxury|        800000|
|             Armenia|Michelle {CraftyM...|        892000|
|               Aruba|         GQ Magazine|        874000|
|           Australia|   Cultura Colectiva|       1000000|
|             Austria|Totschooling | To...|        16100

## Milestone 6 - Querying - Most popular category in different age groups ##

In [0]:
# Setup age group column based on age
df_age_group = pin_geo_user.withColumn(
    "age_group",
    when((col("age") >= 18) & (col("age") <= 24), "18-24")
    .when((col("age") >= 25) & (col("age") <= 35), "25-35")
    .when((col("age") >= 36) & (col("age") <= 50), "36-50")
    .when(col("age") > 50, "50+")
    .otherwise("unknown")  # Unexpected values
)

#df_age_group.show()

# Group by age group and category and count values
category_counts = df_age_group.groupBy("age_group", "category").agg(count("*").alias("category_count"))

# Window function for ranking
win = Window.partitionBy("age_group").orderBy(col("category_count").desc())

# Rank and filter for the top ranked category
category_counts.withColumn("rank", row_number().over(win)) \
.filter(col("rank") == 1) \
.drop("rank") \
.show()


+---------+------------+--------------+
|age_group|    category|category_count|
+---------+------------+--------------+
|    18-24|     tattoos|            27|
|    25-35|         art|            18|
|    36-50|      travel|            17|
|      50+|mens-fashion|             5|
+---------+------------+--------------+



## Milestone 6 - Querying - Median follower count for different age groups ##

In [0]:
# df_age_group set above
# Window function for row numbers and count
win = Window.partitionBy("age_group").orderBy("follower_count")

# row number and total count of rows per age group
df_with_rank = df_age_group \
.withColumn("row_number", expr("ROW_NUMBER() OVER (PARTITION BY age_group ORDER BY follower_count)")) \
.withColumn("total_count", expr("COUNT(*) OVER (PARTITION BY age_group)"))

# Find median rows
df_median = df_with_rank.filter(
    (col("row_number") == (col("total_count") / 2).cast("int")) |  # Lower median row (for even counts)
    (col("row_number") == (col("total_count") / 2 + 1).cast("int"))  # Upper median row
)

# Aggregate to compute the median for each age group
median_result = df_median.groupBy("age_group") \
    .agg(expr("AVG(follower_count)").alias("median_follower_count"))

# Output
median_result.show()

+---------+---------------------+
|age_group|median_follower_count|
+---------+---------------------+
|    18-24|             139000.0|
|    25-35|              27500.0|
|    36-50|               5000.0|
|      50+|               1000.0|
+---------+---------------------+



## Milestone 6 - Querying - How many users have joined each year ##

In [0]:
# Using date_joined to count the number of users who have joined each year
# Get the year from the date_joined column
df_user_with_year = df_user.withColumn("join_year", year(col("date_joined")))

# Group by the year and count the number of users for each year
users_per_year = df_user_with_year.groupBy("join_year").agg(count("*").alias("total_users"))

# Show the result
users_per_year.show()

+---------+-----------+
|join_year|total_users|
+---------+-----------+
|     2015|        184|
|     2016|        235|
|     2017|         81|
+---------+-----------+



## Milestone 6 - Querying - Median count of user based on joining year ##

In [0]:
# Extract year from Timestamp and filter for years between 2015 and 2020
df_post_year = pin_geo_user.withColumn("post_year", year(col("date_joined"))) \
    .filter((col("post_year") >= 2015) & (col("post_year") <= 2020))

# Window function for row numbers and total count per year
win = Window.partitionBy("post_year").orderBy("follower_count")

# Add row_number and total_count columns
df_rank = df_post_year \
    .withColumn("row_number", expr("ROW_NUMBER() OVER (PARTITION BY post_year ORDER BY follower_count)")) \
    .withColumn("total_count", expr("COUNT(*) OVER (PARTITION BY post_year)"))

# Work out median rows
median_df = df_rank.filter(
    (col("row_number") == (col("total_count") / 2).cast("int")) |  # Lower middle row for even count
    (col("row_number") == (col("total_count") / 2 + 1).cast("int"))  # Upper middle row for odd or even count
)

# Aggregate to calculate the median follower count
median_result = median_df.groupBy("post_year") \
    .agg(expr("AVG(follower_count)").alias("median_follower_count"))

# Show the result
median_result.show()

+---------+---------------------+
|post_year|median_follower_count|
+---------+---------------------+
|     2015|             124500.0|
|     2016|              22000.0|
|     2017|               3000.0|
+---------+---------------------+



## Median follower count of users based on their age group##

In [0]:
# df_age_group set above

# Filter for users who joined between 2015 and 2020
df_filtered = df_age_group.withColumn("post_year", year(col("date_joined"))) \
    .filter((col("post_year") >= 2015) & (col("post_year") <= 2020))


# Window function for ranking and counting within age group and year
win = Window.partitionBy("age_group", "post_year").orderBy("follower_count")

df_with_rank = df_filtered.withColumn(
    "row_number", expr("ROW_NUMBER() OVER (PARTITION BY age_group, post_year ORDER BY follower_count)")
).withColumn(
    "total_count", expr("COUNT(*) OVER (PARTITION BY age_group, post_year)")
)

# Median rows
median_df = df_with_rank.filter(
    (col("row_number") == (col("total_count") / 2).cast("int")) |  # Lower median row for even counts
    (col("row_number") == (col("total_count") / 2 + 1).cast("int"))  # Upper median row
)

# Aggregate to calculate the median follower count
median_result = median_df.groupBy("age_group", "post_year") \
    .agg(expr("AVG(follower_count)").alias("median_follower_count"))

# Show the result
median_result.show()

+---------+---------+---------------------+
|age_group|post_year|median_follower_count|
+---------+---------+---------------------+
|    18-24|     2015|             413000.0|
|    18-24|     2016|              46000.0|
|    18-24|     2017|              13500.0|
|    25-35|     2015|              42000.0|
|    25-35|     2016|              27000.0|
|    25-35|     2017|               3500.0|
|    36-50|     2015|               9500.0|
|    36-50|     2016|               6500.0|
|    36-50|     2017|               3000.0|
|      50+|     2015|               7098.0|
|      50+|     2016|                954.0|
|      50+|     2017|                758.5|
+---------+---------+---------------------+

